In [7]:
import os
import sys
from pathlib import Path

# path che vuoi usare come root
ROOT = Path("C:/Users/Dioporco/Desktop/Cognitive Science UOS/4-D/SINDy_Autoencoder")

os.chdir(ROOT)       # cambia la working directory
sys.path.append(str(ROOT))   # così puoi importare model.py, trainer.py, ecc.

print("Current working directory:", os.getcwd())

Current working directory: C:\Users\Dioporco\Desktop\Cognitive Science UOS\4-D\SINDy_Autoencoder


In [8]:
import numpy as np
import pandas as pd
import json

datadir = Path("dataset_2022_silverstone_ham")

X = np.load(datadir/"X.npy")
t = np.load(datadir/"t.npy")
lap_idx = np.load(datadir/"lap_indices.npy")

with open(datadir/"metadata.json") as f:
    meta = json.load(f)

state_cols = meta["state_cols"]
dt = meta["dt"]

scaler = json.load(open(datadir / "scaler.json"))

X_mean = np.array(scaler["X_mean"])
X_scale = np.array(scaler["X_scale"])

X_phys = X * X_scale + X_mean
df_phys = pd.DataFrame(X_phys, columns=state_cols)
df_phys["t"] = t
df_phys["lap"] = lap_idx

df_phys.to_csv(datadir / "dataset_physical_units.csv", index=False)
print(df_phys.head())
print(df_phys.info())
print(X_phys.shape)
print(t.shape)
print(lap_idx.shape)

   Distance       Speed  Throttle  Brake     nGear           RPM            X  \
0  0.000000  252.524998     100.0    0.0  6.000000  10635.925357 -1771.696849   
1  1.405556  252.774999     100.0    0.0  6.000000  10580.175169 -1763.191162   
2  2.813889  253.062112     100.0    0.0  6.012422  10533.118012 -1754.669267   
3  4.247222  253.683230     100.0    0.0  6.136646  10564.298137 -1746.001490   
4  5.680556  254.304348     100.0    0.0  6.260870  10595.478261 -1737.333713   

             Y            Z     t  lap  
0  1189.633010  1963.202931  0.00    0  
1  1201.007901  1963.261456  0.02    0  
2  1212.381744  1963.322393  0.04    0  
3  1223.746160  1963.405044  0.06    0  
4  1235.110575  1963.487695  0.08    0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4526 entries, 0 to 4525
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Distance  4526 non-null   float64
 1   Speed     4526 non-null   float64
 2  

In [9]:
# F1_SINDyAE Notebook - Setup

import numpy as np
import pandas as pd
import json
from pathlib import Path

import torch
import matplotlib.pyplot as plt

from src.model import SINDy_Autoencoder, TimeSeriesDataset   # your model classes
from src.trainer import Trainer_SINDyAE                      # your trainer
from utils.diff_methods import compute_derivatives             # your derivative util

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)



Using device: cpu


In [10]:
# Build fixed-length time windows from the long time series
# Result: X_seq with shape (n_traj, T_window, n_state)

T_window = 200   # sequence length (es. 200 step)
stride = 20      # shift from one window to the following one

N, x_dim = X.shape
print("N samples:", N, "| x_dim:", x_dim)

windows = []
for start in range(0, N - T_window + 1, stride):
    end = start + T_window
    windows.append(X[start:end, :])

X_seq = np.stack(windows, axis=0)   # (n_traj, T_window, x_dim)
print("X_seq shape:", X_seq.shape)

N samples: 4526 | x_dim: 9
X_seq shape: (217, 200, 9)


In [11]:
# Model hyperparameters (puoi adattare ai tuoi)

x_dim = X_seq.shape[-1]   # number of state variables (Distance, Speed, Throttle, ...)
z_dim = 3                 # es: 3 latent dimensions (you choose)

enc_hidden = (128, 64)    # encoder MLP sizes
dec_hidden = (64, 128)    # decoder MLP sizes
poly_order = 3            # polynomial library for SINDy
include_bias = True

model = SINDy_Autoencoder(
    x_dim=x_dim,
    z_dim=z_dim,
    enc_hidden=enc_hidden,
    dec_hidden=dec_hidden,
    poly_order=poly_order,
    include_bias=include_bias,
).to(device)

print(model)

SINDy_Autoencoder(
  (autoencoder): Autoencoder(
    (encoder): Encoder(
      (net): Sequential(
        (0): Linear(in_features=9, out_features=128, bias=True)
        (1): Sigmoid()
        (2): Linear(in_features=128, out_features=64, bias=True)
        (3): Sigmoid()
        (4): Linear(in_features=64, out_features=3, bias=True)
      )
    )
    (decoder): Decoder(
      (net): Sequential(
        (0): Linear(in_features=3, out_features=64, bias=True)
        (1): Sigmoid()
        (2): Linear(in_features=64, out_features=128, bias=True)
        (3): Sigmoid()
        (4): Linear(in_features=128, out_features=9, bias=True)
      )
    )
  )
  (sindy): SINDy_layer()
)


In [ ]:
# Trainer hyperparameters

trainer = Trainer_SINDyAE(
    model=model,
    dt=dt,
    diff_method="finite",      # o il tuo metodo (dipende da diff_methods.py)
    diff_kwargs={},

    batch_size=8,
    shuffle=True,
    device=device,

    lambda_recon=1.0,
    lambda_dx=1e-3,
    lambda_dz=1e-3,

    threshold=0.1,
    threshold_freq=50,
    lr=1e-3,
)

print("Trainer ready.")

[INIT] Using device: cpu
[INIT] Model first param device: cpu
Trainer ready.


In [ ]:
# Train the model on the F1 sequences

n_epochs = 500
log_every = 50

trainer.fit(
    X_seq,
    n_epochs=n_epochs,
    log_every=log_every,
)

In [ ]:
# Inspect training history

hist = pd.DataFrame(trainer.history_loss)
display(hist.tail())

plt.figure(figsize=(6,4))
plt.plot(hist["loss"], label="Total loss")
plt.plot(hist["L_recon"], label="Recon")
plt.plot(hist["L_dx"], label="L_dx")
plt.plot(hist["L_dz"], label="L_dz")
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("loss (log)")
plt.legend()
plt.title("Training losses")
plt.tight_layout()
plt.show()